In [196]:
# jupyter nbconvert --to script rpi_bird_detection.ipynb

In [173]:
import cv2
import numpy as np
import tensorflow as tf  # ON PC, we can use full TensorFlow
import matplotlib.pyplot as plt
import time
import os
import subprocess

# import tflite_runtime.interpreter as tflite
tflite = tf.lite

In [175]:
# load model
interpreter = tf.lite.Interpreter(model_path="bird_detection_model_v1.tflite")
interpreter.allocate_tensors()

In [177]:
# --- Config ---
INPUT_SIZE = (180, 180)
ROWS, COLS = 8, 8
CONFIDENCE_THRESHOLD = 0.50
MODEL_PATH = "bird_detection_model_v1.tflite"
SOUND_FILENAME = 'alert.wav'
CLASS_NAMES = ['no_bird', 'with_bird']
LOOP_INTERVAL = 3

In [179]:
# --- Load TFLite Model ---
interpreter = tflite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [191]:
def save_detected_bird(tile_img, cam_id, tile_id, confidence):
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder = "detections"
    os.makedirs(folder, exist_ok=True)
    filename = f"{folder}/cam{cam_id}_tile{tile_id}_{int(confidence*100)}_{now}.jpg"
    cv2.imwrite(filename, tile_img)
    print(f"📸 Saved detection: {filename}")

def predict_tile(tile):
    img = cv2.resize(tile, INPUT_SIZE).astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    pred_class = np.argmax(output[0])
    confidence = output[0][pred_class]
    return CLASS_NAMES[pred_class], confidence

def play_sound_deterent():
    print("🔊 Playing sound...")
    SOUND_FILENAME = "alert.wav"
    sound_path = os.path.join(os.getcwd(), SOUND_FILENAME)
    return subprocess.Popen(["aplay", sound_path])

def deploy_drone():
    print("🚁 Drone: Starting mission...")
    time.sleep(10)

def respond_to_bird():
    sound_proc = play_sound_deterent()
    deploy_drone()
    sound_proc.terminate()  # stop the sound (optional)

def preprocess_frame_to_tiles(frame):
    tiles = []
    coords = []
    height, width = frame.shape[:2]
    tile_h, tile_w = height // ROWS, width // COLS

    for r in range(ROWS):
        for c in range(COLS):
            y1, y2 = r * tile_h, (r + 1) * tile_h
            x1, x2 = c * tile_w, (c + 1) * tile_w
            tile = frame[y1:y2, x1:x2]
            tiles.append(tile)
            coords.append((r, c, x1, y1))
    return tiles, coords

In [193]:
# Setup cameras
capture_one = cv2.VideoCapture(0)
capture_two = cv2.VideoCapture(1)

for cap in [capture_one, capture_two]:
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

print("🚀 Detection started. Press Q to quit.")

🚀 Detection started. Press Q to quit.


In [194]:
loop_count = 0

while True:
    for cam_id, cap in enumerate([capture_one, capture_two]):
        ret, frame = cap.read()
        if not ret:
            print(f"❌ Failed to read from camera {cam_id}")
            continue

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        tiles, coords = preprocess_frame_to_tiles(frame_rgb)

        for i, tile in enumerate(tiles):
            label, conf = predict_tile(tile)
            r, c, x, y = coords[i]
            print(f"[CAM {cam_id}] Tile {i} [{r},{c}] → {label} ({conf:.2f})")

            if label == "with_bird" and conf >= CONFIDENCE_THRESHOLD:
                print(f"🚨 [CAM {cam_id}] BIRD DETECTED at tile [{r},{c}]!")
                respond_to_bird()
                save_detected_bird(tiles[i], cam_id, i, confidence)
                break

    if cv2.waitKey(1) & 0xFF == ord("q"):
        print("🛑 Stopping...")
        break

    print("sleepp...")
    time.sleep(LOOP_INTERVAL)
    loop_count = loop_count + 1

    if loop_count >= 3:
        print("break the loop...")
        break

capture_one.release()
capture_two.release()
cv2.destroyAllWindows()

[CAM 0] Tile 0 [0,0] → no_bird (0.81)
[CAM 0] Tile 1 [0,1] → no_bird (0.82)
[CAM 0] Tile 2 [0,2] → no_bird (0.84)
[CAM 0] Tile 3 [0,3] → no_bird (0.66)
[CAM 0] Tile 4 [0,4] → no_bird (0.70)
[CAM 0] Tile 5 [0,5] → no_bird (0.72)
[CAM 0] Tile 6 [0,6] → no_bird (0.51)
[CAM 0] Tile 7 [0,7] → with_bird (0.54)
🚨 [CAM 0] BIRD DETECTED at tile [0,7]!
🔊 Playing sound...


FileNotFoundError: [WinError 2] The system cannot find the file specified